In [72]:
import streamlit as st
import pandas as pd
import requests
import ast

In [ ]:
nested_egg = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/290a91fb-8186-486d-992e-985839352809/data/latest')
loot = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/2208cc91-e624-4b40-b666-f65c8b6405f8/data/latest')
eggs = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/ee319c5f-4f03-41b5-b607-99f306fe1e2f/data/latest')
meteor_dust = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/8eebec3c-db0e-4051-a5c2-50a5deb1070b/data/latest')
meteors = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/b9778a1d-9b63-4ba5-a48f-7b28569b5603/data/latest')
#IT IS ABSOLUTELY CRUCIAL THAT YOU DO NOT CHANGE THE ORIGINAL NAMES OF THE DATAFRAME COLUMNS ON VELOCITY

#MUST STORE IN A DICTIONARY 
current_nft_dict = {'nested_egg':nested_egg, 'loot':loot, 'eggs':eggs, 'meteor_dust':meteor_dust, 'meteors':meteors}



In [ ]:
nft_luna_price_df = []
for nft in current_nft_dict:
    #renames columns from Flipside dataframes according to NFT
    nft_luna_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_LUNA_PRICE']].rename({'NFT_LUNA_PRICE':f'{nft}'}, axis = 1))
nft_luna_price_df = pd.concat(nft_luna_price_df)
nft_luna_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_luna_price_df['BLOCK_TIMESTAMP'])
nft_luna_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
nft_luna_price_df.index = nft_luna_price_df.index.round('D')

# START HERE

In [ ]:
nft_luna_price_df = []
for nft in current_nft_dict:
    #renames columns from Flipside dataframes according to NFT
    nft_luna_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_TYPE', 'NFT_LUNA_PRICE', 'NFT_UST_PRICE_AT_PURCHASE']])

nft_luna_price_df = pd.concat(nft_luna_price_df)
nft_luna_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_luna_price_df['BLOCK_TIMESTAMP'])
nft_luna_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
nft_luna_price_df.index = nft_luna_price_df.index.round('D')
nft_luna_price_df


In [ ]:
total_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).sum().rename(columns={'NFT_LUNA_PRICE':'TOTAL_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'TOTAL_UST'})
average_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).mean().rename(columns={'NFT_LUNA_PRICE':'AVERAGE_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'AVERAGE_UST'})
tx_count_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).count().rename(columns={'NFT_LUNA_PRICE':'TRANSACTION_COUNT'})
tx_count_df = tx_count_df['TRANSACTION_COUNT']
min_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).min().rename(columns={'NFT_LUNA_PRICE':'MIN_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MIN_UST'})
max_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).max().rename(columns={'NFT_LUNA_PRICE':'MAX_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MAX_UST'})
median_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).median().rename(columns={'NFT_LUNA_PRICE':'MEDIAN_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MEDIAN_UST'})
cum_sum = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).sum().rename(columns={'NFT_LUNA_PRICE':'TOTAL_LUNA_CUMULATIVE', 'NFT_UST_PRICE_AT_PURCHASE':'TOTAL_UST_CUMULATIVE'}).cumsum(axis=0)


master_df = pd.concat([total_df, tx_count_df, average_df, min_df, max_df, median_df, cum_sum], axis = 1)
master_df

# RARITY TABLE - Meteors, Eggs, Dust


In [ ]:
# nft_luna_price_df = []
# for nft in current_nft_dict:
#     #renames columns from Flipside dataframes according to NFT
#     nft_luna_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_LUNA_PRICE']].rename({'NFT_LUNA_PRICE':f'{nft}'}, axis = 1))
# nft_luna_price_df = pd.concat(nft_luna_price_df)
# nft_luna_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_luna_price_df['BLOCK_TIMESTAMP'])
# nft_luna_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
# nft_luna_price_df.index = nft_luna_price_df.index.round('D')

#meteors
meteor_rarity = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/5d7b0f96-a537-46e5-9b3c-d5145072279f/data/latest')
meteor_rarity_merge = pd.merge(meteors, meteor_rarity[['TOKEN_ID', 'RARITY']], on='TOKEN_ID')
egg_rarity = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/284600cc-7b4c-4d47-ab50-dbdd528daf93/data/latest')
egg_rarity_merge = pd.merge(eggs, egg_rarity[['TOKEN_ID', 'RARITY']], on='TOKEN_ID')
dust_rarity = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/f4b78c34-07ec-4f5e-8099-7e6db2db24a9/data/latest')
dust_rarity_merge = pd.merge(meteor_dust, dust_rarity[['TOKEN_ID', 'RARITY']], on='TOKEN_ID')




# RARITY - Nested Eggs, Loot

In [ ]:
#Alternative using swagger; this method takes an exceptionally large amount of time
def swagger_app(num):
        rarity = requests.get(f"https://fcd.terra.dev/wasm/contracts/terra1vhuyuwwr4rkdpez5f5lmuqavut28h5dt29rpn6/store?query_msg=%7B%22nft_info%22:%7B%22token_id%22:%22{num}%22%7D%7D").json()
        return rarity.get('result').get('extension').get('attributes')[1].get('value')
        #print(num)

nested_egg['RARITY'] = nested_egg['TOKEN_ID'].apply(lambda x: swagger_app(x))


In [ ]:
#for nested eggs
#This should be the faster version pulling directly from the server set up to scrape RE
nested_rarity = pd.read_csv('http://165.22.125.123/nested_egg_nfts.csv ')
nested_rarity_merge = pd.merge(nested_egg, nested_rarity[['token_id', 'traits']], left_on='TOKEN_ID', right_on='token_id', how='inner')
nested_rarity_merge['RARITY'] = nested_rarity_merge['traits'].apply(lambda x: ast.literal_eval(x).get('Rarity'))
nested_rarity_merge.pop('traits')
nested_rarity_merge

In [67]:
loot_rarity = pd.read_csv('http://165.22.125.123/loot_nfts.csv')
loot_rarity_merge = pd.merge(loot, loot_rarity[['token_id', 'traits']], left_on='TOKEN_ID', right_on='token_id', how='inner')
loot_rarity_merge['RARITY'] = loot_rarity_merge['traits'].apply(lambda x: ast.literal_eval(x).get('Rarity'))
loot_rarity_merge.pop('traits')
loot_rarity_merge

,BLOCK_TIMESTAMP,HOUR,LUNA_UST_PRICE,NFT_LUNA_PRICE,NFT_TYPE,NFT_UST_PRICE_AT_PURCHASE,TOKEN_ID,TX_ID,token_id,RARITY
0,2022-02-01 18:05:46.834,2022-02-01 18:00:00.000,52.204693,1.75,Levana Loot,91.358213,3110,530DC5DACA625F01A66CFC865EEA717DC0367A4DB21458...,3110,None
1,2022-02-01 17:36:32.132,2022-02-01 17:00:00.000,52.207602,6.12,Levana Loot,319.510523,625,F9648A102B52659D339E1B7823DD5C5E124CDB93C5E51D...,625,None
2,2022-02-01 17:11:26.029,2022-02-01 17:00:00.000,52.207602,6.07,Levana Loot,316.900143,1590,5A39F8EAEEC27731E60EAA3255A84E547956F1CF11AC66...,1590,None
3,2022-02-01 16:39:16.438,2022-02-01 16:00:00.000,52.041715,5.63,Levana Loot,292.994855,3878,67E40CBE29C96F831C45EC6B4F3E7BC699BBD15152B577...,3878,None
4,2022-02-01 16:34:49.595,2022-02-01 16:00:00.000,52.041715,6.00,Levana Loot,312.250290,2024,90D30E3F0192EE28BBD76AF3BD163320FC41416C541872...,2024,None
...,...,...,...,...,...,...,...,...,...,...
638,2022-01-12 13:36:49.066,2022-01-12 13:00:00.000,78.392752,7.00,Levana Loot,548.749264,336,98AA8E00EF2F994929ED619EAA924B60C86A48411BE0B5...,336,None
639,2022-01-12 12:28:57.408,2022-01-12 12:00:00.000,78.293709,2.50,Levana Loot,195.734272,20,34DA05F39CFECBDB2584F7B43A04E261FDECF4C6502447...,20,None
640,2022-01-12 12:29:40.436,2022-01-12 12:00:00.000,78.293709,2.50,Levana Loot,195.734272,62,939D8F43C21F1A4E846C829E81D01B35CAD4F46B18242D...,62,None
641,2022-01-12 12:45:08.249,2022-01-12 12:00:00.000,78.293709,2.50,Levana Loot,195.734272,129,8D108BCECDD68B4D5403A03262514CF893387B92F2B970...,129,None


In [75]:
group_master = ['BLOCK_TIMESTAMP']
meteors = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/b9778a1d-9b63-4ba5-a48f-7b28569b5603/data/latest')
meteors_rarity = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/284600cc-7b4c-4d47-ab50-dbdd528daf93/data/latest')
meteors_rarity_merge = pd.merge(meteors, meteors_rarity[group_master], on='TOKEN_ID')

meteors_rarity_merge = meteors_rarity_merge[['BLOCK_TIMESTAMP', 'RARITY', 'NFT_LUNA_PRICE', 'NFT_UST_PRICE_AT_PURCHASE']]
meteors_rarity_merge['BLOCK_TIMESTAMP'] = pd.to_datetime(meteors_rarity_merge['BLOCK_TIMESTAMP'])
meteors_rarity_merge.set_index('BLOCK_TIMESTAMP', inplace = True)
meteors_rarity_merge.index = meteors_rarity_merge.index.round('D')

total_df = meteors_rarity_merge.groupby(group_master).sum().rename(columns={'NFT_LUNA_PRICE':'TOTAL_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'TOTAL_UST'})
average_df = meteors_rarity_merge.groupby(group_master).mean().rename(columns={'NFT_LUNA_PRICE':'AVERAGE_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'AVERAGE_UST'})
tx_count_df = meteors_rarity_merge.groupby(group_master).count().rename(columns={'NFT_LUNA_PRICE':'TRANSACTION_COUNT'})
tx_count_df = tx_count_df['TRANSACTION_COUNT']
min_df = meteors_rarity_merge.groupby(group_master).min().rename(columns={'NFT_LUNA_PRICE':'MIN_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MIN_UST'})
max_df = meteors_rarity_merge.groupby(group_master).max().rename(columns={'NFT_LUNA_PRICE':'MAX_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MAX_UST'})
median_df = meteors_rarity_merge.groupby(group_master).median().rename(columns={'NFT_LUNA_PRICE':'MEDIAN_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MEDIAN_UST'})
cum_sum = meteors_rarity_merge.groupby(group_master).sum().rename(columns={'NFT_LUNA_PRICE':'TOTAL_LUNA_CUMULATIVE', 'NFT_UST_PRICE_AT_PURCHASE':'TOTAL_UST_CUMULATIVE'}).cumsum(axis=0)

meteors_master = pd.concat([total_df, tx_count_df, average_df, min_df, max_df, median_df, cum_sum], axis = 1).reset_index().sort_values(by=group_master, ascending=False)
meteors_master

,BLOCK_TIMESTAMP,TOTAL_LUNA,TOTAL_UST,TRANSACTION_COUNT,AVERAGE_LUNA,AVERAGE_UST,RARITY,MIN_LUNA,MIN_UST,RARITY,MAX_LUNA,MAX_UST,MEDIAN_LUNA,MEDIAN_UST,TOTAL_LUNA_CUMULATIVE,TOTAL_UST_CUMULATIVE
78,2022-02-01,46.540,2434.301965,4,11.635000,608.575491,Common,6.85,354.263027,Rare,23.00,1191.373058,8.345,444.332939,14712.073788,769488.570095
77,2022-01-31,29.300,1335.814593,2,14.650000,667.907296,Common,7.30,335.384831,Rare,22.00,1000.429762,14.650,667.907296,14665.533788,767054.268130
76,2022-01-30,6.890,354.765033,1,6.890000,354.765033,Common,6.89,354.765033,Common,6.89,354.765033,6.890,354.765033,14636.233788,765718.453538
75,2022-01-29,35.490,1746.727305,3,11.830000,582.242435,Common,5.99,310.755772,Rare,22.50,1095.214390,7.000,340.757143,14629.343788,765363.688505
74,2022-01-28,28.750,1653.072853,2,14.375000,826.536426,Common,6.25,336.928810,Rare,22.50,1316.144043,14.375,826.536426,14593.853788,763616.961200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2021-11-15,390.670,19787.035365,40,9.766750,494.675884,Ancient,1.00,50.795981,Rare,150.00,7639.221720,2.600,131.895697,2349.209000,119074.586554
3,2021-11-14,347.510,17492.017524,23,15.109130,760.522501,Ancient,1.24,63.143337,Rare,179.00,8735.005144,2.000,105.053126,1958.539000,99287.551189
2,2021-11-13,101.229,4959.449481,19,5.327842,261.023657,Ancient,1.59,78.531544,Rare,9.90,488.561600,6.650,328.890969,1611.029000,81795.533665
1,2021-11-12,1185.350,60116.491989,59,20.090678,1018.923593,Ancient,1.10,56.162096,Rare,155.00,7787.968202,7.000,357.395156,1509.800000,76836.084184


In [ ]:
nft_ust_price_df = []
for nft in current_nft_dict:
    #renames columns from Flipside dataframes according to NFT
    nft_ust_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_UST_PRICE_AT_PURCHASE']].rename({'NFT_UST_PRICE_AT_PURCHASE':f'{nft}_UST_PRICE'}, axis = 1))
nft_ust_price_df = pd.concat(nft_ust_price_df)
nft_ust_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_ust_price_df['BLOCK_TIMESTAMP'])
nft_ust_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
nft_ust_price_df.index = nft_ust_price_df.index.round('D')
nft_ust_price_df.groupby('BLOCK_TIMESTAMP').sum()

In [71]:
nested_egg = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/290a91fb-8186-486d-992e-985839352809/data/latest')
nested_rarity = pd.read_csv('http://165.22.125.123/nested_egg_nfts.csv')
nested_rarity_merge = pd.merge(nested_egg, nested_rarity[['token_id', 'traits']], left_on='TOKEN_ID', right_on='token_id', how='inner')
nested_rarity_merge['RARITY'] = nested_rarity_merge['traits'].apply(lambda x: ast.literal_eval(x).get('Rarity'))
nested_rarity_merge.pop('traits')

group_master = ['BLOCK_TIMESTAMP']

nested_rarity_merge = nested_rarity_merge[['BLOCK_TIMESTAMP', 'NFT_LUNA_PRICE', 'NFT_UST_PRICE_AT_PURCHASE']]
nested_rarity_merge['BLOCK_TIMESTAMP'] = pd.to_datetime(nested_rarity_merge['BLOCK_TIMESTAMP'])
nested_rarity_merge.set_index('BLOCK_TIMESTAMP', inplace = True)
nested_rarity_merge.index = nested_rarity_merge.index.round('D')

total_df = nested_rarity_merge.groupby(group_master).sum().rename(columns={'NFT_LUNA_PRICE':'TOTAL_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'TOTAL_UST'})
average_df = nested_rarity_merge.groupby(group_master).mean().rename(columns={'NFT_LUNA_PRICE':'AVERAGE_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'AVERAGE_UST'})
tx_count_df = nested_rarity_merge.groupby(group_master).count().rename(columns={'NFT_LUNA_PRICE':'TRANSACTION_COUNT'})
tx_count_df = tx_count_df['TRANSACTION_COUNT']
min_df = nested_rarity_merge.groupby(group_master).min().rename(columns={'NFT_LUNA_PRICE':'MIN_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MIN_UST'})
max_df = nested_rarity_merge.groupby(group_master).max().rename(columns={'NFT_LUNA_PRICE':'MAX_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MAX_UST'})
median_df = nested_rarity_merge.groupby(group_master).median().rename(columns={'NFT_LUNA_PRICE':'MEDIAN_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MEDIAN_UST'})
cum_sum = nested_rarity_merge.groupby(group_master).sum().rename(columns={'NFT_LUNA_PRICE':'TOTAL_LUNA_CUMULATIVE', 'NFT_UST_PRICE_AT_PURCHASE':'TOTAL_UST_CUMULATIVE'}).cumsum(axis=0)

nested_master = pd.concat([total_df, tx_count_df, average_df, min_df, max_df, median_df, cum_sum], axis = 1).reset_index().sort_values(by=group_master, ascending=False)
nested_master

,BLOCK_TIMESTAMP,TOTAL_LUNA,TOTAL_UST,TRANSACTION_COUNT,AVERAGE_LUNA,AVERAGE_UST,MIN_LUNA,MIN_UST,MAX_LUNA,MAX_UST,MEDIAN_LUNA,MEDIAN_UST,TOTAL_LUNA_CUMULATIVE,TOTAL_UST_CUMULATIVE
20,2022-02-02,92.090,4788.265028,5,18.418000,957.653006,6.99,361.763566,40.00,2085.309165,12.600,655.725608,6494.413,438693.687517
19,2022-02-01,712.240,37129.060703,34,20.948235,1092.031197,5.50,275.882123,200.00,10343.446056,6.150,322.005397,6402.323,433905.422489
18,2022-01-31,489.150,23229.697428,15,32.610000,1548.646495,5.00,246.079980,170.00,8495.527716,6.000,280.966968,5690.083,396776.361786
17,2022-01-30,127.840,6628.401073,19,6.728421,348.863214,5.25,274.830214,12.50,642.477332,5.750,291.799528,5200.933,373546.664359
16,2022-01-29,260.650,12911.684152,14,18.617857,922.263154,5.25,272.159499,85.00,4137.476585,6.150,318.562213,5073.093,366918.263286
15,2022-01-28,733.815,38260.209880,12,61.151250,3188.350823,5.90,322.455030,400.00,20146.851333,11.750,672.324282,4812.443,354006.579134
14,2022-01-27,155.080,9236.058513,28,5.538571,329.859233,4.00,251.464809,11.00,639.395557,4.970,289.963305,4078.628,315746.369254
13,2022-01-26,427.150,26932.261334,11,38.831818,2448.387394,4.00,255.464921,345.00,21737.353963,8.300,522.956632,3923.548,306510.310741
12,2022-01-25,81.630,5379.386262,7,11.661429,768.483752,5.00,305.072592,50.00,3317.746311,5.380,353.822734,3496.398,279578.049407
11,2022-01-24,84.510,5691.292012,11,7.682727,517.390183,5.00,333.725184,13.00,893.414753,6.120,412.997449,3414.768,274198.663146


In [ ]:
tx_count = pd.DataFrame(nft_ust_price_df.apply(lambda x: (x > 0).sum(), axis =  1).groupby('BLOCK_TIMESTAMP').sum()).reset_index()
pd.DataFrame(tx_count.rename(columns={0:"Transaction Count"})['Transaction Count'])

In [ ]:
nft_luna_price_df.index = nft_luna_price_df.index.round('H')
nft_luna_price_df.groupby('BLOCK_TIMESTAMP').sum()